# 3.6 Planes de Ejecución - Leer y Optimizar Query Plans

## 🎯 ¿Para qué?

---

## 🎯 Objetivos de Aprendizaje
- Interpretar planes gráficos: identificar index seek vs. scan, joins anidados vs. hash.
- Detectar operadores problemáticos (table scan en millones de filas, implicit conversions).
- Usar query hints y forzar planes cuando el optimizador falla.

## 🧰 Prerrequisitos
- Haber completado `05_transacciones_bloqueos.ipynb`.
- Conocer índices clustered/nonclustered y estadísticas de tablas.
- Acceso a herramientas de análisis de planes (SSMS, Azure Data Studio).

---

## 📚 Conceptos Clave
- **Plan de ejecución**: árbol de operadores que SQL Server usa para ejecutar query; generado por optimizador basándose en estadísticas.
- **Index seek**: búsqueda eficiente por índice (operación logarítmica).
- **Table/Index scan**: recorrido completo de tabla/índice (operación lineal, costosa).
- **Key lookup**: ida extra a clustered index para traer columnas no incluidas en nonclustered index.
- **Operadores de JOIN**: Nested Loop (pequeños conjuntos), Hash Match (grandes conjuntos), Merge Join (datos pre-ordenados).

> ⚠️ Plan estimado puede diferir del plan actual si estadísticas están desactualizadas; siempre validar con plan actual en volúmenes reales.

In [ ]:
-- Consulta que genera plan de ejecución interesante
SET SHOWPLAN_TEXT ON; -- Muestra plan en texto (alternativa a gráfico)
GO

SELECT c.nombre, p.nombre AS producto, SUM(v.cantidad) AS unidades
FROM dbo.fact_ventas v
INNER JOIN dbo.dim_clientes c ON v.cliente_id = c.cliente_id
INNER JOIN dbo.dim_productos p ON v.producto_id = p.producto_id
WHERE v.fecha >= '2024-01-01'
GROUP BY c.nombre, p.nombre
ORDER BY unidades DESC;
GO

SET SHOWPLAN_TEXT OFF;
GO

/*
📊 Elementos clave del plan (lee de derecha a izquierda, abajo hacia arriba):

1. **Table Scan vs Index Seek**:
   - Table Scan (🔴): lee TODA la tabla, lento en tablas grandes
   - Index Seek (🟢): usa índice, solo lee filas necesarias
   - Index Scan (🟠): escanea índice completo (mejor que table scan pero no óptimo)

2. **Operadores de JOIN**:
   - Nested Loop (🟢): rápido para datasets pequeños
   - Hash Match (🟠): para tablas grandes sin índices
   - Merge Join (🟢): eficiente si ambas entradas ordenadas

3. **Sort**: costoso, intenta evitarlo con índices ordenados

4. **Costo relativo**: % al lado de cada operador (busca los >20%)
*/

## Patrones comunes en planes problemáticos

### 🔴 **Table Scan en tabla grande**
```
Clustered Index Scan (fact_ventas) Cost: 85%
```
**Problema**: lee 10M filas sin índice  
**Solución**: crear índice en columna filtrada en WHERE

### 🔴 **Key Lookup** (Bookmark Lookup)
```
Index Seek → Key Lookup → Nested Loops Cost: 60%
```
**Problema**: índice no cubre todas las columnas del SELECT, hace idas extras a la tabla  
**Solución**: agregar columnas a INCLUDE en el índice

### 🔴 **Hash Match en JOINs pequeños**
```
Hash Match (Inner Join) Cost: 45%
```
**Problema**: SQL Server eligió hash en vez de nested loop (¿estadísticas desactualizadas?)  
**Solución**: UPDATE STATISTICS o crear índice en columna de JOIN

### 🔴 **Sort costoso**
```
Sort (ORDER BY ...) Cost: 30%
```
**Problema**: no hay índice ordenado por la columna de ORDER BY  
**Solución**: índice con esa columna al final (permite sort-free execution)

## 🟢 Ejercicio Básico
Ejecuta esta consulta con plan de ejecución activado:
```sql
SELECT * FROM fact_ventas WHERE cliente_id = 5;
```
¿Hace Table Scan o Index Seek? Si hace scan, ¿qué índice recomendarías?

## 🟠 Ejercicio Intermedio
Analiza el plan de esta query:
```sql
SELECT c.nombre, COUNT(*) AS num_ventas
FROM fact_ventas v
JOIN dim_clientes c ON v.cliente_id = c.cliente_id
WHERE v.fecha >= '2024-01-01'
GROUP BY c.nombre
ORDER BY num_ventas DESC;
```
Identifica:
1. ¿Qué operador de JOIN usa? (Nested Loop, Hash Match, Merge Join)
2. ¿Hay Sort al final? ¿Cuánto cuesta?
3. Propón un índice que elimine el Sort

## 🔴 Ejercicio Avanzado
Optimiza esta query problemática analizando su plan:
```sql
SELECT p.nombre, AVG(v.cantidad) AS promedio
FROM dim_productos p
LEFT JOIN fact_ventas v ON p.producto_id = v.producto_id
WHERE v.fecha >= '2023-01-01'
GROUP BY p.nombre
HAVING AVG(v.cantidad) > 10;
```
**Problemas a detectar:**
- ¿Hace Table Scan en qué tabla?
- ¿El filtro WHERE se aplica antes o después del JOIN? (mira orden de operadores)
- ¿Hay Key Lookups?

**Entregables:**
- Screenshot del plan con operadores costosos marcados
- Query refactorizada
- 2-3 índices propuestos con justificación

---

## Hints (pistas al optimizador)

A veces quieres forzar un comportamiento (uso avanzado):

```sql
-- Forzar uso de índice específico
SELECT * FROM fact_ventas WITH (INDEX(idx_ventas_fecha))
WHERE fecha >= '2024-01-01';

-- Forzar tipo de JOIN
SELECT * FROM tabla1 t1
INNER HASH JOIN tabla2 t2 ON t1.id = t2.id;

-- Evitar paralelismo (útil en debug)
SELECT * FROM tabla OPTION (MAXDOP 1);
```

⚠️ **Cuidado**: hints pueden hacer que el optimizador elija plan subóptimo si los datos cambian.

---

## Errores Comunes

❌ **No mirar el plan**: "está lento, agrego índice random" → puede empeorar
❌ **Solo mirar costo total**: un operador al 90% es el cuello de botella, no el todo
❌ **Ignorar advertencias**: iconos amarillos (⚠️) indican problemas (estadísticas faltantes, implicit conversions)
❌ **Hints en producción sin justificar**: el optimizador suele ser más inteligente que tú
❌ **No actualizar estadísticas**: plan se basa en stats antiguas, elige mal

**Siguiente:** `07_query_tuning.ipynb` → técnicas avanzadas de optimización

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.

---
## Navegación
[⬅️ Anterior](05_transacciones_bloqueos.ipynb) | [Siguiente ➡️](07_query_tuning.ipynb)
---
